# Fine-tuning OpenVLA for Humanoid Robotics

This notebook demonstrates how to fine-tune the OpenVLA (Open Vision-Language-Action) model for humanoid robotics tasks. OpenVLA combines visual understanding, language processing, and action generation in a unified framework.

## Learning Objectives
- Understand the OpenVLA architecture and its components
- Prepare robotics datasets for VLA training
- Fine-tune OpenVLA on custom humanoid manipulation tasks
- Evaluate the fine-tuned model's performance
- Deploy the model for real-world humanoid robot control

In [ ]:
# Install required packages
!pip install torch torchvision torchaudio
!pip install transformers
!pip install datasets
!pip install accelerate
!pip install bitsandbytes
!pip install peft
!pip install openvla

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from transformers import AutoTokenizer, AutoModel, AutoProcessor
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import json
import os
from tqdm import tqdm
import random

## 1. Understanding OpenVLA Architecture

OpenVLA is built upon the foundation of vision-language models like CLIP, extended to include action prediction capabilities. The architecture consists of:

1. **Vision Encoder**: Processes visual input (images)
2. **Language Encoder**: Processes textual input (commands)
3. **Fusion Module**: Combines visual and linguistic features
4. **Action Decoder**: Generates robot actions based on fused features

In [ ]:
# Load the pre-trained OpenVLA model
from openvla import load

# Load the pre-trained OpenVLA model
# Note: In practice, you would load the specific OpenVLA variant
def load_openvla_model(model_path="openvla/openvla-7b"):
    """Load the OpenVLA model"""
    try:
        model = load(model_path)
        print(f"Successfully loaded OpenVLA model from {model_path}")
        return model
    except Exception as e:
        print(f"Error loading model: {e}")
        # For demonstration, create a simplified model
        return SimpleOpenVLA()

class SimpleOpenVLA(nn.Module):
    """Simplified OpenVLA model for demonstration"""
    def __init__(self, vision_dim=512, lang_dim=512, action_dim=7):
        super().__init__()
        
        # Vision encoder (using a pre-trained CNN)
        from torchvision.models import resnet18
        self.vision_encoder = resnet18(pretrained=True)
        self.vision_encoder.fc = nn.Identity()  # Remove final layer
        self.vision_proj = nn.Linear(512, vision_dim)
        
        # Language encoder (simplified)
        self.lang_embedding = nn.Embedding(10000, lang_dim)  # Vocabulary size
        self.lang_encoder = nn.LSTM(lang_dim, lang_dim, batch_first=True)
        
        # Fusion layer
        self.fusion = nn.Sequential(
            nn.Linear(vision_dim + lang_dim, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
        )
        
        # Action decoder
        self.action_head = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, action_dim)
        )
        
    def forward(self, images, text_ids):
        # Encode vision
        vision_features = self.vision_encoder(images)
        vision_features = self.vision_proj(vision_features)
        
        # Encode language
        lang_embeddings = self.lang_embedding(text_ids)
        lang_output, (hidden, _) = self.lang_encoder(lang_embeddings)
        lang_features = hidden[-1]  # Use last hidden state
        
        # Fuse features
        fused_features = torch.cat([vision_features, lang_features], dim=1)
        fused_features = self.fusion(fused_features)
        
        # Generate actions
        actions = self.action_head(fused_features)
        
        return actions

# Load model
model = load_openvla_model()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(f"Model loaded on {device}")

## 2. Preparing Robotics Datasets

For fine-tuning OpenVLA, we need datasets containing:
- Images of the robot and environment
- Natural language commands
- Corresponding robot actions

We'll create a synthetic dataset for demonstration purposes, but in practice, you would use real robot data.

In [ ]:
# Create a synthetic robotics dataset
class RoboticsDataset(Dataset):
    def __init__(self, num_samples=1000):
        self.num_samples = num_samples
        self.samples = []
        
        # Define possible commands and objects
        self.commands = [
            "pick up the red cup",
            "move to the blue box",
            "grasp the green object",
            "navigate to the table",
            "pick up the small ball",
            "move toward the chair",
            "grasp the yellow item",
            "go to the left"
        ]
        
        # Generate synthetic data
        for i in range(num_samples):
            # Create a random image tensor (simulating camera input)
            image = torch.randn(3, 224, 224)  # RGB image 224x224
            
            # Select a random command
            command = random.choice(self.commands)
            
            # Generate corresponding action (simplified)
            # In reality, this would come from robot demonstrations
            action = torch.randn(7)  # 7-DoF action (e.g., joint positions)
            
            self.samples.append({
                'image': image,
                'command': command,
                'action': action
            })
    
    def __len__(self):
        return self.num_samples
    
    def __getitem__(self, idx):
        sample = self.samples[idx]
        
        # Tokenize command (simplified)
        command = sample['command']
        tokens = self.tokenize_command(command)
        
        return {
            'image': sample['image'],
            'tokens': tokens,
            'action': sample['action']
        }
    
    def tokenize_command(self, command):
        """Simple tokenization for demonstration"""
        # In practice, use a proper tokenizer
        words = command.lower().split()
        tokens = [hash(word) % 10000 for word in words]  # Simple hashing
        tokens = tokens[:20]  # Limit sequence length
        tokens += [0] * (20 - len(tokens))  # Pad to fixed length
        return torch.tensor(tokens)

# Create dataset
dataset = RoboticsDataset(num_samples=2000)
print(f"Dataset created with {len(dataset)} samples")

# Split into train and validation
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")

## 3. Fine-tuning Configuration

We'll set up the training configuration including optimizer, loss function, and hyperparameters.

In [ ]:
# Training configuration
batch_size = 16
learning_rate = 1e-4
num_epochs = 10
warmup_steps = 100

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Loss function
criterion = nn.MSELoss()

# Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# Learning rate scheduler
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)

print("Training configuration set up")

## 4. Training Loop

Now we'll implement the training loop for fine-tuning the OpenVLA model.

In [ ]:
def train_epoch(model, dataloader, criterion, optimizer, device):
    """Train for one epoch"""
    model.train()
    total_loss = 0
    num_batches = 0
    
    progress_bar = tqdm(dataloader, desc="Training")
    
    for batch in progress_bar:
        # Move data to device
        images = batch['image'].to(device)
        tokens = batch['tokens'].to(device)
        actions = batch['action'].to(device)
        
        # Forward pass
        predicted_actions = model(images, tokens)
        
        # Calculate loss
        loss = criterion(predicted_actions, actions)
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        
        total_loss += loss.item()
        num_batches += 1
        
        # Update progress bar
        progress_bar.set_postfix({'Loss': f'{loss.item():.4f}'})
    
    avg_loss = total_loss / num_batches
    return avg_loss

def validate_epoch(model, dataloader, criterion, device):
    """Validate for one epoch"""
    model.eval()
    total_loss = 0
    num_batches = 0
    
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Validation"):
            # Move data to device
            images = batch['image'].to(device)
            tokens = batch['tokens'].to(device)
            actions = batch['action'].to(device)
            
            # Forward pass
            predicted_actions = model(images, tokens)
            
            # Calculate loss
            loss = criterion(predicted_actions, actions)
            
            total_loss += loss.item()
            num_batches += 1
    
    avg_loss = total_loss / num_batches
    return avg_loss

# Training loop
train_losses = []
val_losses = []

best_val_loss = float('inf')
best_model_state = None

print("Starting training...")
for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    
    # Train
    train_loss = train_epoch(model, train_loader, criterion, optimizer, device)
    train_losses.append(train_loss)
    
    # Validate
    val_loss = validate_epoch(model, val_loader, criterion, device)
    val_losses.append(val_loss)
    
    # Update learning rate
    scheduler.step()
    
    print(f"Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")
    
    # Save best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model_state = model.state_dict().copy()
        print(f"New best model saved with validation loss: {val_loss:.4f}")

print("\nTraining completed!")

## 5. Model Evaluation

Let's evaluate the fine-tuned model's performance.

In [ ]:
import matplotlib.pyplot as plt

# Plot training curves
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_losses, label='Training Loss')
plt.title('Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

# Load best model for evaluation
if best_model_state is not None:
    model.load_state_dict(best_model_state)
    print("Best model loaded for evaluation")

# Evaluate on a few examples
model.eval()
eval_samples = [val_dataset[i] for i in range(min(5, len(val_dataset)))]

print("\nEvaluation Examples:")
for i, sample in enumerate(eval_samples):
    image = sample['image'].unsqueeze(0).to(device)  # Add batch dimension
    tokens = sample['tokens'].unsqueeze(0).to(device)  # Add batch dimension
    true_action = sample['action']
    
    with torch.no_grad():
        predicted_action = model(image, tokens)
    
    # Calculate error
    error = torch.mean((predicted_action.cpu() - true_action) ** 2)
    
    print(f"Example {i+1}:")
    print(f"  Command: {dataset.samples[val_dataset.indices[i]]['command']}")
    print(f"  True Action: {true_action[:3].numpy()}")  # Show first 3 dims
    print(f"  Predicted Action: {predicted_action[0, :3].cpu().numpy()}")
    print(f"  MSE Error: {error.item():.4f}")
    print()

## 6. Model Deployment for Robotics

Now let's prepare the model for deployment in a robotics environment using ROS 2.

In [ ]:
# Save the fine-tuned model
model_save_path = "/content/drive/MyDrive/openvla_finetuned.pth"  # Adjust path as needed
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'train_losses': train_losses,
    'val_losses': val_losses,
    'epoch': num_epochs,
}, model_save_path)

print(f"Model saved to {model_save_path}")

# Create a simplified ROS 2 node for model inference
# This is a conceptual example - in practice, you'd use the ROS 2 Python API

class OpenVLAROSNode:
    def __init__(self, model_path):
        # Load the fine-tuned model
        self.model = SimpleOpenVLA()  # Use the same architecture
        checkpoint = torch.load(model_path)
        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.model.eval()
        
        # Move to device
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model.to(self.device)
        
        print("OpenVLA model loaded for ROS deployment")
    
    def preprocess_image(self, image):
        """Preprocess image for model input"""
        # Resize and normalize image
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                                std=[0.229, 0.224, 0.225])
        ])
        return transform(image).unsqueeze(0)  # Add batch dimension
    
    def tokenize_command(self, command):
        """Tokenize natural language command"""
        # In practice, use the same tokenizer as during training
        words = command.lower().split()
        tokens = [hash(word) % 10000 for word in words]
        tokens = tokens[:20]  # Limit sequence length
        tokens += [0] * (20 - len(tokens))  # Pad to fixed length
        return torch.tensor(tokens).unsqueeze(0)  # Add batch dimension
    
    def predict_action(self, image, command):
        """Predict robot action from image and command"""
        # Preprocess inputs
        image_tensor = self.preprocess_image(image).to(self.device)
        command_tokens = self.tokenize_command(command).to(self.device)
        
        # Get prediction
        with torch.no_grad():
            action = self.model(image_tensor, command_tokens)
        
        return action.cpu().numpy()[0]  # Remove batch dimension
    
    def process_robot_command(self, camera_image, natural_language_command):
        """Process a complete robot command"""
        try:
            # Get action prediction
            predicted_action = self.predict_action(camera_image, natural_language_command)
            
            print(f"Command: '{natural_language_command}'")
            print(f"Predicted action: {predicted_action[:3]}")  # Show first 3 dims
            
            # In a real ROS node, you would publish this action to the robot
            # For example, to a joint trajectory controller or Cartesian controller
            return predicted_action
            
        except Exception as e:
            print(f"Error processing command: {e}")
            return None

# Example usage of the ROS node
# In practice, this would be integrated into a ROS 2 package
if os.path.exists(model_save_path):
    ros_node = OpenVLAROSNode(model_save_path)
    
    # Simulate processing a command
    # In reality, this would come from camera and voice input
    dummy_image = Image.new('RGB', (640, 480), color='red')
    command = "pick up the red cup"
    
    action = ros_node.process_robot_command(dummy_image, command)
    if action is not None:
        print(f"Successfully processed command, predicted action shape: {action.shape}")

## 7. Advanced Fine-tuning Techniques

Let's explore some advanced techniques for improving OpenVLA fine-tuning.

In [ ]:
# Advanced fine-tuning with Parameter-Efficient Fine-Tuning (PEFT)
from peft import LoraConfig, get_peft_model, TaskType

# Define LoRA configuration for efficient fine-tuning
def setup_lora_finetuning(model, lora_r=8, lora_alpha=16, lora_dropout=0.1):
    """Set up LoRA fine-tuning for the model"""
    # In practice, LoRA would be applied to specific layers
    # This is a simplified example
    
    # Apply LoRA to linear layers
    for name, module in model.named_modules():
        if isinstance(module, nn.Linear):
            # Replace with LoRA layer (conceptual)
            # In practice, use PEFT library
            pass
    
    return model

# Alternative: Using Hugging Face PEFT library (if available)
def setup_hf_peft(model):
    """Set up PEFT using Hugging Face library"""
    try:
        # Define LoRA config
        peft_config = LoraConfig(
            task_type=TaskType.CAUSAL_LM,  # Adjust based on model type
            inference_mode=False,
            r=8,
            lora_alpha=32,
            lora_dropout=0.1,
            target_modules=["q_proj", "v_proj", "k_proj", "o_proj"]  # Example for transformers
        )
        
        # Get PEFT model
        peft_model = get_peft_model(model, peft_config)
        return peft_model
    except ImportError:
        print("PEFT library not available, using full fine-tuning")
        return model

# Apply PEFT if needed
# model = setup_hf_peft(model)

print("PEFT setup demonstrated (conceptual)")

## 8. Conclusion and Next Steps

In this notebook, we've demonstrated how to fine-tune an OpenVLA model for humanoid robotics tasks. We covered:

1. Understanding OpenVLA architecture
2. Preparing robotics datasets
3. Setting up fine-tuning configuration
4. Training the model
5. Evaluating performance
6. Preparing for ROS 2 deployment
7. Advanced fine-tuning techniques

### Next Steps for Real Implementation:

1. **Collect Real Robot Data**: Gather actual robot demonstrations with synchronized images, commands, and actions
2. **Scale Dataset**: Create a larger, more diverse dataset with various objects, environments, and tasks
3. **Improve Model Architecture**: Consider more sophisticated fusion mechanisms or multi-task learning
4. **Implement Safety Mechanisms**: Add safety checks and validation before executing predicted actions
5. **Deploy on Real Robot**: Integrate with your humanoid robot platform
6. **Continuous Learning**: Implement online learning to adapt to new tasks and environments

In [ ]:
# Final summary
print("=== OpenVLA Fine-tuning Summary ===")
print(f"Total epochs: {num_epochs}")
print(f"Final training loss: {train_losses[-1]:.4f}")
print(f"Final validation loss: {val_losses[-1]:.4f}")
print(f"Best validation loss: {best_val_loss:.4f}")
print(f"Total training samples: {len(train_dataset)}")
print(f"Total validation samples: {len(val_dataset)}")
print("Model saved successfully for deployment!")

print("\nNext Steps:")
steps = [
    "1. Collect real robot data for more robust training",
    "2. Experiment with different architectures and hyperparameters",
    "3. Implement safety validation before action execution",
    "4. Deploy on your humanoid robot platform",
    "5. Evaluate in real-world scenarios"
]

for step in steps:
    print(f"  {step}")